# Analysis of Interferometric Wavefront Data

In this example, we will see how to use prysm to almost entirely supplant the software that comes with a commerical interferometer to analyze the wavefront of an optic.  We begin by importing the relevant classes and setting some aesthetics for matplotlib.

In [ ]:
from prysm import Interferogram, FringeZernike, sample_files, zernikefit

from matplotlib import pyplot as plt
plt.style.use('bmh')

We point prysm to the file, create a new interferogram, mask it to a circular region 100 mm across, subtract piston, tip/tilt and power, and evalute the PV and RMS wavefront error. We also plot the wavefront.

In [ ]:
p = sample_files('dat')  # sample Zygo .dat file, will be downloaded on demand and saved locally
i = Interferogram.from_zygo_dat(p)
i.crop().mask('circle', 40).crop()
i.remove_piston_tiptilt_power()
print(i.pv, i.rms)
i.plot2d(clim=100, interpolation='bilinear')  # +/- 100 nm
plt.grid(False)

The interferogram is cropped twice – once to enclose the valid data, then again to apply a mask centered on that region. For relatively conventional interferometry, you may want to stop here. If you want to use a different unit, that is easy enough,

In [ ]:
i.change_z_unit('waves')
1/i.pv, 1/i.rms  # print reciprocal -- "one over xxx waves"

There is no need to crop again since the outer bound has not changed. Perhaps you wish to evaluated the RMS within the 1 - 10 mm spatial periods,

In [ ]:
i.change_z_unit('nm')
i.fill()
i.bandlimited_rms(1,10)

This value is derived from the PSD, so you must call fill first. Do not worry about the corners of the array containing data - it will be windowed out. If you do this on a part which has a central obscuration or otherwise departs from being a circle or rectangle, the result will be correct.

If you wish to decompose the wavefront into Zernike polynomials, that is easy enough.

In [ ]:
# do this on data which has not been filled to avoid errors introduced by the fill value.
coefficients = zernikefit(i.phase, terms=36, norm=True, map_='fringe')
fz = FringeZernike(coefficients, dia=i.diameter, units=i.units, norm=True)
print(fz)

This print might be a bit daunting, one may prefer to see the top few terms by magnitude,

In [ ]:
fz.top_n(5)

or a barplot of all terms,

In [ ]:
fz.barplot_magnitudes(orientation='v', sort=True)

The sample data has a circular clear aperture, but if it had a central obscuration (such as transmitted wavefront data for a telescope) that would be easy to mask too.  Here we will build a composite mask for the data as if it were a telescope with an annual aperture disrupted by a spider:

In [ ]:
from prysm.geometry import circle, inverted_circle, generate_spider

outer = circle(i.samples_x, radius=1) # radius has units of array semidiameter
inner = inverted_circle(i.samples_x, radius=0.35)

# width has units of arydiam, or pixels if arydiam=None
spider = generate_spider(vanes=3, width=0.5, rotation=90, arydiam=i.diameter, samples=i.samples_x)
mask = outer * inner * spider

i.mask(mask)
i.plot2d(clim=100)  # +/- 100 nm
plt.grid(False)